In [ ]:
#Import function library path
from IPython.display import display
from uuid import uuid1
import inspect
import ctypes
import os
from robot import *
from camera import *
from frame import *
import threading
import time
from red_object import *
import math

myobject = RedObject()
mycamera = RCamera(255,255)
status = 0

myrobot = Robit()
def jetbot_motion_center_x():
    #red object detected the robot move
    if myobject.pos_x_px >= 148:
        #red object detected on right of camera rotate right
        myrobot.turn_right()
    elif myobject.pos_x_px <= 108:
        #red object detected on left of camera rotate left
        myrobot.turn_left()
    else :
        #red object on camera center stop moving straight
        myrobot.stop()
        
    time.sleep(.01)



def findminrow():
    xpxcm = 34.5929693
    if myframe.color_y > 127:
        yobj = myframe.color_y - 127.
        xcm = xpxcm * 128./yobj
        print("Real time distance (cm)" + xcm)
        print("Pixels below center: ",yobj)
        myobject.pos_z = xcm


myframe = Frame(mycamera.getDisplay(),myobject.color_lower,myobject.color_upper)
display(myframe.image)
def qcheck(thread2,thread3):
    if(thread2.is_alive() == False and thread3.is_alive() == False): ## Think i can do something here where I put this in a thread and .join() this thread instead of thread4. This way this thread is always getting reset and thread4 gets reset within it
        if myframe.status == 1:
            thread2 = threading.Thread(target=jetbot_motion_center_x)
            thread3 = threading.Thread(target=findminrow)
            myobject.pos_x_px = myframe.color_x
            myobject.pos_y_px = myframe.color_y
            thread2.start()
            thread3.start()

def dothecamthing():
    while 1:
        newframe = mycamera.getDisplay()
        myframe.setFrame(newframe,myobject.color_lower,myobject.color_upper)

thread4 = threading.Thread(target=dothecamthing,daemon=True)
thread4.start()

thread2 = threading.Thread()
thread3 = threading.Thread()

while 1:
    if(thread2.is_alive() == False and thread3.is_alive() == False): 
        if myframe.status == 1:
            thread2 = threading.Thread(target=jetbot_motion_center_x)
            thread3 = threading.Thread(target=findminrow)
            myobject.pos_x_px = myframe.color_x
            myobject.pos_y_px = myframe.color_y
            thread2.start()
            thread3.start()
    
    

In [ ]:
# is red obj pres or not. 1 when red object present else 0
status = 0 
# lower and upper bound of BGR values the robit will detect
global color_lower
global color_upper
# xy coordinate of center of detected color and radius of detected color
global color_x, color_y, color_radius

#image = widgets.Image(format='jpeg', width=600, height=224)  # this width and height doesn't necessarily have to match the camera
global minrow 
global newmask 
global maskdf
minrow = 0
newmask = 0
maskdf = pd.DataFrame(np.zeros((255,255)))


color_x = color_y = color_radius = 0

display(color_image)

# assign values to color bounds. Current value assigned 2/10/23 is [0,0,100] __> [75,75,255] which covers area of red
color_lower=np.array([0,0,100])
color_upper = np.array([75, 75, 255])

def jetbot_motion_center_x():
    while 1:
        if status == 0:
            #no red object then the robot does not move
            robot.stop
            time.sleep(.01)
        else:
            #red object detected the robot move
            if color_x >= 148:
                #red object detected on right of camera rotate right
                setleft = .75
                setright = -.75 
            elif color_x <= 108:
                #red object detected on left of camera rotate left
                setleft = -.75
                setright = .75
            else :
                #red object on camera center stop moving straight
                setleft = 0
                setright = 0
                
            robot.set_motors(setleft,setright)
            time.sleep(.01)

thread2 = threading.Thread(target=jetbot_motion_center_x)
thread2.setDaemon(True)
thread2.start()
global xpxcm
xpxcm = 34.5929693
def findminrow():
    global xpxcm
    global maskdf
    global newmask
    global minrow
    while 1:
        if newmask == 1:
            maskdf = pd.DataFrame(mask)
            for c in range(108,149):
                for r in range(127,255):
                    if maskdf.iloc[r,c] == 255:
                        if r > minrow:
                            minrow = r
            if minrow > 127:
                yobj = minrow - 127.
                xcm = xpxcm * 128./yobj
                print(xcm)
                print("yobj: ",yobj)
            minrow = 0
            newmask = 0
        else:
            pass
thread3 = threading.Thread(target=findminrow)
thread3.setDaemon(True)
thread3.start()
while 1:
    #capture camera frame
    frame = camera.value
    # print the contents of the ndarray frame which stores the rbg values of the camera frame
    """print(frame)"""
    #resize camera frame idk y
    frame = cv2.resize(frame, (255, 255))
    #add gaussian blur. this has yet to improve performace but it came stock
    frame_=cv2.GaussianBlur(frame,(1,1),0)  
    mask=cv2.inRange(frame,color_lower,color_upper)
    # print the contents of the ndarray mask which stores 255 if a pixel is within the range specified or 0 if the pixel is not
    """print(mask)"""
    # search mask for the lowest row where there is an element with 255 as its value
    maskdf = pd.DataFrame(mask)
    newmask = 1
    mask=cv2.erode(mask,None,iterations=2)
    mask=cv2.dilate(mask,None,iterations=2)
    mask=cv2.GaussianBlur(mask,(1,1),0)     
    cnts=cv2.findContours(mask.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2]
    """print(cnts)"""
    """print(cnts[0][0][0][0])
    print(cnts[0][0][0][1])
    print(cnts[0][1][0][0])
    print(cnts[0][1][0][1])
    print(cnts[0][2][0][0])
    print(cnts[0][2][0][1])
    print(cnts[0][3][0][0])
    print(cnts[0][3][0][1])"""
    #display x and y coordinate. only used for understanding and debugging
    """print("coor x: " + str(color_x) + "\tcoor y: " + str(color_y))"""
    if len(cnts)>0:
        cnt = max (cnts,key=cv2.contourArea)
        (color_x,color_y),color_radius=cv2.minEnclosingCircle(cnt)
        if color_radius > 2:
            # Mark the detected color with circle
            #left is x=0; right is x = 255
            #high is y=0; low is y = 255
            status = 1
            cv2.circle(frame,(int(color_x),int(color_y)),int(color_radius),(255,0,255),2)  #might uncommment
            #end mark detected color with circle
            # Mark the detected color with rectangle
            """rot_rect = cv2.minAreaRect(cnt)
            box = cv2.boxPoints(rot_rect)
            box = np.int0(box)
            cv2.drawContours(frame,[box],0,(0,0,0,),2)"""
            #end mark detected color with rectangle
    else:
        status = 0
    # Real-time return of image data for display
    color_image.value = bgr8_to_jpeg(frame)